In [306]:
import pandas as pd
import matplotlib.pyplot as plt

In [307]:
df = pd.read_csv("parsed_logs_fi.csv")

In [308]:
df["config"].unique()

array(['c10_res44_test_01_bn-relu_base', 'c10_res44_test_02_bn-relu6',
       'c10_res44_test_02_bn-relu6_base', 'c10_res44_test_02_relu6-bn',
       'c10_res44_test_02_relu6-bn_nanfilter',
       'c10_res44_test_02_gelu6_nans'], dtype=object)

In [309]:
configs = dict(
    # Baseline C100
    BaseC100="c100_res44_test_01_bn-relu_base.yaml",
    # "c100_res44_test_02_bn-relu6.yaml",
    # Order inversion with relu6 C100
    OrderInvC100="c100_res44_test_02_relu6-bn.yaml",
    # "c100_res44_test_02_bn-relu6_base.yaml",
    BaseBnRelu6C100="c100_res44_test_02_bn-relu6_base.yaml",
    BnRelu6C100="c100_res44_test_02_bn-relu6.yaml",
    # Order inversion + nan filter + Relu6
    OrderINanRelu6C100="c100_res44_test_02_relu6-bn_nanfilter.yaml",
    # Gelu and nan C100
    GeluNanC100="c100_res44_test_02_gelu6_nans.yaml",

    # Baseline C10
    BaseC10="c10_res44_test_01_bn-relu_base.yaml",
    # "c10_res44_test_02_bn-relu6.yaml",
    BnRelu6C10="c10_res44_test_02_bn-relu6.yaml",
    # Order inversion with relu6 C10
    OrderInvC10="c10_res44_test_02_relu6-bn.yaml",
    # "c10_res44_test_02_bn-relu6_base.yaml",
    BaseBnRelu6C10="c10_res44_test_02_bn-relu6_base.yaml",
    OrderINanRelu6C10="c10_res44_test_02_relu6-bn_nanfilter.yaml",
    # Gelu and nan C10
    GeluNanC10="c10_res44_test_02_gelu6_nans.yaml",

    MobileNetC100="cifar100_mobilenetv2_x1_4.yaml",
    MobileNetC10="cifar10_mobilenetv2_x1_4.yaml"
)
configs_inv = {v.replace(".yaml", ""): k for k, v in configs.items()}

In [310]:
configs_df = pd.DataFrame([configs]).T.reset_index().rename(columns={0:"YAML", "index": "config"})

In [311]:
configs_df

,config,YAML
0,BaseC100,c100_res44_test_01_bn-relu_base.yaml
1,OrderInvC100,c100_res44_test_02_relu6-bn.yaml
2,BaseBnRelu6C100,c100_res44_test_02_bn-relu6_base.yaml
3,BnRelu6C100,c100_res44_test_02_bn-relu6.yaml
4,OrderINanRelu6C100,c100_res44_test_02_relu6-bn_nanfilter.yaml
5,GeluNanC100,c100_res44_test_02_gelu6_nans.yaml
6,BaseC10,c10_res44_test_01_bn-relu_base.yaml
7,BnRelu6C10,c10_res44_test_02_bn-relu6.yaml
8,OrderInvC10,c10_res44_test_02_relu6-bn.yaml
9,BaseBnRelu6C10,c10_res44_test_02_bn-relu6_base.yaml


In [312]:
df["config"] = df["config"].apply(lambda x: configs_inv[x])

In [313]:
df["count"] = 1

In [314]:
df[df["sdc"] == 0]

,config,ecc,hostname,fi_model,group,has_end,it,ker_time,acc_time,ker_err,acc_err,sdc,critical_sdc,count
0,BaseC10,OFF,caroltitaninria,FLIP_SINGLE_BIT,fp32,1,0,0.0,0.0,0,0,0.0,0.0,1
1,BaseC10,OFF,caroltitaninria,FLIP_SINGLE_BIT,fp32,1,0,0.0,0.0,0,0,0.0,0.0,1
4,BaseC10,OFF,caroltitaninria,FLIP_SINGLE_BIT,fp32,1,0,0.0,0.0,0,0,0.0,0.0,1
5,BaseC10,OFF,caroltitaninria,FLIP_SINGLE_BIT,fp32,1,0,0.0,0.0,0,0,0.0,0.0,1
6,BaseC10,OFF,caroltitaninria,FLIP_SINGLE_BIT,fp32,1,0,0.0,0.0,0,0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11993,GeluNanC10,OFF,caroltitaninria,WARP_ZERO_VALUE,fp32,1,0,0.0,0.0,0,0,0.0,0.0,1
11995,GeluNanC10,OFF,caroltitaninria,WARP_ZERO_VALUE,fp32,1,0,0.0,0.0,0,0,0.0,0.0,1
11997,GeluNanC10,OFF,caroltitaninria,WARP_ZERO_VALUE,fp32,1,0,0.0,0.0,0,0,0.0,0.0,1
11998,GeluNanC10,OFF,caroltitaninria,WARP_ZERO_VALUE,fp32,1,0,0.0,0.0,0,0,0.0,0.0,1


In [315]:
to_save = df.groupby(["hostname", "config", "fi_model"]).sum()[["has_end", "sdc", "critical_sdc", "count"]].reset_index()

In [316]:
NEW_NAMES = dict(
        # Baseline C100
    BaseC100="Base",
    # "c100_res44_test_02_bn-relu6.yaml",
    # Order inversion with relu6 C100
    OrderInvC100="OrdInv",
    # "c100_res44_test_02_bn-relu6_base.yaml",
    BaseBnRelu6C100="BaseR6",
    BnRelu6C100="BnR6",
    # Order inversion + nan filter + Relu6
    OrderINanRelu6C100="OrdInvNanR6",
    # Gelu and nan C100
    GeluNanC100="Gelu6Nan",
)
NEW_NAMES.update({k.replace("C100", "C10"): v for k,v in NEW_NAMES.items()})

In [317]:
to_save

,hostname,config,fi_model,has_end,sdc,critical_sdc,count
0,caroltitaninria,BaseBnRelu6C10,FLIP_SINGLE_BIT,400,124.0,1.0,400
1,caroltitaninria,BaseBnRelu6C10,RANDOM_VALUE,400,179.0,2.0,400
2,caroltitaninria,BaseBnRelu6C10,WARP_RANDOM_VALUE,400,246.0,12.0,400
3,caroltitaninria,BaseBnRelu6C10,WARP_ZERO_VALUE,400,251.0,0.0,400
4,caroltitaninria,BaseBnRelu6C10,ZERO_VALUE,400,158.0,0.0,400
5,caroltitaninria,BaseC10,FLIP_SINGLE_BIT,400,138.0,11.0,400
6,caroltitaninria,BaseC10,RANDOM_VALUE,400,164.0,48.0,400
7,caroltitaninria,BaseC10,WARP_RANDOM_VALUE,400,225.0,154.0,400
8,caroltitaninria,BaseC10,WARP_ZERO_VALUE,400,228.0,0.0,400
9,caroltitaninria,BaseC10,ZERO_VALUE,400,172.0,0.0,400


In [318]:
FM_NAMES = dict(FLIP_SINGLE_BIT="Single Bit", ZERO_VALUE="Zero Val",  WARP_ZERO_VALUE="Warp Z.V.", RANDOM_VALUE="Rand Val",  WARP_RANDOM_VALUE="Warp R. V.")

In [319]:
to_save["dataset"] = to_save["config"].apply(lambda x: "Cifar 100" if "C100" in x else "Cifar 10")
to_save["config"] = to_save["config"].apply(lambda x: NEW_NAMES[x])
to_save["fi_model"] = to_save["fi_model"].apply(lambda x: FM_NAMES[x])


In [320]:
to_save

,hostname,config,fi_model,has_end,sdc,critical_sdc,count,dataset
0,caroltitaninria,BaseR6,Single Bit,400,124.0,1.0,400,Cifar 10
1,caroltitaninria,BaseR6,Rand Val,400,179.0,2.0,400,Cifar 10
2,caroltitaninria,BaseR6,Warp R. V.,400,246.0,12.0,400,Cifar 10
3,caroltitaninria,BaseR6,Warp Z.V.,400,251.0,0.0,400,Cifar 10
4,caroltitaninria,BaseR6,Zero Val,400,158.0,0.0,400,Cifar 10
5,caroltitaninria,Base,Single Bit,400,138.0,11.0,400,Cifar 10
6,caroltitaninria,Base,Rand Val,400,164.0,48.0,400,Cifar 10
7,caroltitaninria,Base,Warp R. V.,400,225.0,154.0,400,Cifar 10
8,caroltitaninria,Base,Warp Z.V.,400,228.0,0.0,400,Cifar 10
9,caroltitaninria,Base,Zero Val,400,172.0,0.0,400,Cifar 10


In [321]:
to_save_tail = to_save.groupby(["hostname", "dataset", "config"]).sum().reset_index()

In [322]:
to_save_tail["fi_model"] = "Overall"

In [323]:
to_save_gp = pd.concat([to_save, to_save_tail])

In [326]:
to_save_gp = to_save_gp.groupby(["hostname", "dataset", "fi_model", "config"]).sum()

In [327]:
to_save_gp.to_excel("/home/fernando/git_research/dsn_2023_diehardnet/data/c10_avf_400faults.xlsx")